In [2]:
%pwd

'c:\\Users\\akish.pothuri\\python\\SentimentAnalysis\\SentimentAPI\\research'

In [3]:
import os
os.chdir("../")
%pwd

'c:\\Users\\akish.pothuri\\python\\SentimentAnalysis\\SentimentAPI'

In [4]:
from sklearn.model_selection import train_test_split
import keras

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    METRIC_FILE: Path
    model: Path

In [6]:
from Sentiment.constants import *
from Sentiment.utils.common import read_yaml, create_directories

In [16]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            METRIC_FILE = config.METRIC_FILE,
            model = config.model
        )

        return model_evaluation_config

In [8]:
import os
from Sentiment import logger
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score

In [9]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def loadModel(self):
        model = keras.models.load_model(self.config.model)
        return model
    
    def predictTest(self,testSentences,model):
        return model.predict(testSentences)
    
    def EvaluationMetrics(self,predictions,testLabels):
        try:
            accuracyScore = accuracy_score(testLabels,predictions)
            recallScore = recall_score(testLabels,predictions)
            precisionScore = precision_score(testLabels,predictions)
            f1Score = f1_score(testLabels,predictions)
            return accuracyScore,recallScore,precisionScore,f1Score
        except Exception as e:
            raise e
        
    def ModelEvaluationStatus(self,accuracyScore,recallScore,precisionScore,f1Score):
        try:
            validation_status = None

            all_files = os.listdir(os.path.join("artifacts","model_evaluation"))

            for file in all_files:
                if file not in self.config.ALL_REQUIRED_FILES:
                    Evaluation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Evaluation status: {Evaluation_status}")
                else:
                    Evaluation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Evaluation status: {Evaluation_status}"+",")
                        f.write(f"accuracyScore: {accuracyScore}"+",")
                        f.write(f"recallScore: {recallScore}"+",")
                        f.write(f"precisionScore: {precisionScore}"+",")
                        f.write(f"f1Score: {f1Score}"+",")

            return validation_status
        
        except Exception as e:
            raise e

In [11]:
from Sentiment.components.model_trainer import ModelTrainer
# from Sentiment.config.configuration import mo

In [13]:
from Sentiment.config.configuration import ConfigurationManager

In [17]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    logger.info(f"data loading Started")
    sentiData = model_trainer_config.loadData()
    logger.info(f"data loading Completed")
    logger.info(f"data Lowercase convertion Started")
    sentiData = model_trainer_config.convertToLower(sentiData)
    logger.info(f"data Lowercase convertion Completed")
    logger.info(f"data Splitting Started")
    trainSentences, testSentences, trainLabels, testLabels = model_trainer_config.splitData(sentiData)
    logger.info(f"data Splitting Completed")
    logger.info(f"Model Loading")
    model = model_evaluation_config.loadModel()
    logger.info(f"Model Loaded")
    logger.info(f"Predicting Test Data")
    predictions = model_evaluation_config.predictTest(model,testSentences)
    logger.info(f"Predicting Test Data Completed")
    logger.info(f"EvaluationMetrics Started")
    accuracyScore,recallScore,precisionScore,f1Score = model_evaluation_config.EvaluationMetrics(predictions,testLabels)
    logger.info(f"EvaluationMetrics Completed")
    model_evaluation_config.ModelEvaluationStatus(accuracyScore,recallScore,precisionScore,f1Score)
    logger.info(f"EvaluationMetrics Saved")
except Exception as e:
    raise e

[2024-01-05 00:35:29,137: INFO: common: yaml file: {yaml_file}]
[2024-01-05 00:35:29,155: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-05 00:35:29,158: INFO: common: yaml file: {yaml_file}]
[2024-01-05 00:35:29,162: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-05 00:35:29,170: INFO: common: created directory at: artifacts]


AttributeError: 'ConfigurationManager' object has no attribute 'get_model_trainer_config'

In [ ]:
tr